# Modelisation

In [2]:
%load_ext autoreload
%autoreload 2

import warnings
import pandas as pd

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 5000)
pd.set_option("display.max_rows", 101)
pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))

In [6]:
# Load the data
from sklearn.model_selection import train_test_split

from utils import RANDOM_STATE, TARGET, TEST_SIZE


X = pd.read_csv("./data/kaggle_train_set.csv")
y = X.pop(TARGET)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
    )

print(f"Le train set contient: {X_train.shape} élements")
print(f"Le test set contient: {X_test.shape} élements")

Le train set contient: (944, 15) élements
Le test set contient: (236, 15) élements
